시간당 내부 평균온도: 4도~40도

시간당 내부 평균습도: 0% ~ 100%

시간당 내부 평균 co2 농도 : 0ppm ~ 1200 ppm

시간당 평균 EC : 0 ~ 8

시간당 분무량 : 0 ~ 3000 / 일간 누적 분무량 0 ~ 72,000

시간당 백색광량 : 0 ~ 120,000 / 일간 누적 백색광량 0 ~ 2,880,000

시간당 적색광량 : 0 ~ 120,000 / 일간 누적 적색광량 0 ~ 2,880,000

시간당 청색광량 : 0 ~ 120,000 / 일간 누적 청색광량 0 ~ 2,880,000

시간당 총광량 : 0 ~ 120,000 / 일간 누적 총광량 0 ~ 2,880,000

In [ ]:
# 나눔고딕 폰트 설치 및 설정
!apt-get update -qq
!apt-get install fonts-nanum -qq
!fc-cache -fv
!rm ~/.cache/matplotlib -rf


In [2]:

import matplotlib.pyplot as plt

# 폰트 설정
import matplotlib.font_manager as fm

# font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
# fontprop = fm.FontProperties(fname=font_path, size=10)
# plt.rcParams['font.family'] = 'NanumGothic'
# plt.rcParams['axes.unicode_minus'] = False

# 폰트 설정
font_path='/System/Library/Fonts/AppleGothic.ttf'
fontprop = fm.FontProperties(fname=font_path, size=10)
plt.rcParams['font.family'] = 'AppleGothic'  # macOS
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
# 필수 라이브러리 설치
# !pip install xgboost lightgbm scikit-learn tensorflow matplotlib seaborn --quiet

import pandas as pd
import numpy as np
import glob
import os

# 1. 모든 CSV 파일 읽어오기 및 합치기
def load_all_csv_files(folder_path):
    """폴더 내 모든 CSV 파일을 읽어서 하나의 DataFrame으로 합침"""
    csv_files = glob.glob(os.path.join(folder_path, "*.csv"))
    dataframes = []
    
    for file in sorted(csv_files):
        df = pd.read_csv(file)
        # 파일명에서 CASE 번호 추출 후 컬럼으로 추가
        case_num = os.path.basename(file).replace('.csv', '')
        df['CASE'] = case_num
        dataframes.append(df)
    
    return pd.concat(dataframes, ignore_index=True)

# train_input 폴더의 모든 CSV 파일 읽기
train_input_path = './상추_생육_예측/train_input'
X = load_all_csv_files(train_input_path)

# train_target 폴더의 모든 CSV 파일 읽기  
train_target_path = './상추_생육_예측/train_target'
y_df = load_all_csv_files(train_target_path)

print("입력 데이터 shape:", X.shape)
print("타겟 데이터 shape:", y_df.shape)
print("입력 데이터 컬럼:", X.columns.tolist())
print("타겟 데이터 컬럼:", y_df.columns.tolist())

if 'obs_time' in X.columns:
    # obs_time에서 시간(hh) 추출하여 숫자 컬럼으로 추가
    X['time'] = X['obs_time'].str.split(':').str[0].astype(int)

# 타겟 컬럼이 여러 개라면, 'predicted_weight_g'만 사용
target_col = 'predicted_weight_g'


# CASE와 DAT를 기준으로 필터링된 X와 y_df 병합
df = pd.merge(X, y_df[['CASE', 'DAT', target_col]], on=['CASE', 'DAT'], how='left')

print("병합된 데이터 shape:", df.shape)
print("병합된 데이터 첫 5행:")
print(df.head())

입력 데이터 shape: (18816, 17)
타겟 데이터 shape: (784, 3)
입력 데이터 컬럼: ['DAT', 'obs_time', '내부온도관측치', '내부습도관측치', 'co2관측치', 'ec관측치', '시간당분무량', '일간누적분무량', '시간당백색광량', '일간누적백색광량', '시간당적색광량', '일간누적적색광량', '시간당청색광량', '일간누적청색광량', '시간당총광량', '일간누적총광량', 'CASE']
타겟 데이터 컬럼: ['DAT', 'predicted_weight_g', 'CASE']

hour 컬럼 추가 후 X shape: (18816, 18)
hour 컬럼 unique values: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]

필터링된 입력 데이터 shape: (588, 17)
병합된 데이터 shape: (18816, 19)
병합된 데이터 첫 5행:
   DAT obs_time    내부온도관측치    내부습도관측치      co2관측치     ec관측치  시간당분무량  일간누적분무량  \
0    0    00:00  25.300000  81.835000  536.016667  1.407439     0.0      0.0   
1    0    01:00  25.680357  81.264286  528.696429  1.409003   126.0    126.0   
2    0    02:00  25.273333  81.471666  532.833333  1.406913     0.0    126.0   
3    0    03:00  25.355000  81.398334  545.566667  1.406689   126.0    252.0   
4    0    04:00  25.391667  81.483333  558.583333  1.411070     0.0    252.0   

   시간당백색광량  일간

In [8]:
def check_nan():
    # 결측값 확인
    missing_values = df.isnull().sum()
    missing_columns = missing_values[missing_values > 0]

    print(f"전체 결측값 개수: {missing_values.sum()}")
    if len(missing_columns) > 0:
        print("결측값이 있는 컬럼:")
        for col, count in missing_columns.items():
            percentage = (count / len(df)) * 100
            print(f"  - {col}: {count}개 ({percentage:.1f}%)")
    else:
        print("결측값이 있는 컬럼: 없음")

check_nan()

# 2. 결측치/이상치 처리 (간단 예시)
# df = df.dropna()  # 결측치 행 제거 (상황에 따라 평균/중앙값 대체 가능)


전체 결측값 개수: 672
결측값이 있는 컬럼:
  - predicted_weight_g: 672개 (3.6%)


In [ ]:

# 3. 특성과 타겟 분리
y = df[target_col]
X = df.drop([target_col], axis=1)

# 문자형 피처 라벨 인코딩
from sklearn.preprocessing import LabelEncoder

for col in X.select_dtypes(include='object').columns:
    X[col] = LabelEncoder().fit_transform(X[col])

# 4. 학습/테스트 분할
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# 5. 공통 평가 및 시각화 함수 (회귀)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def print_regression_metrics(y_true, y_pred, model_name="Model"):
    print(f"\n[{model_name} 평가]")
    print("MAE :", mean_absolute_error(y_true, y_pred))
    print("RMSE:", np.sqrt(mean_squared_error(y_true, y_pred)))
    print("R2  :", r2_score(y_true, y_pred))

def plot_regression_results(y_true, y_pred, model_name="Model"):
    plt.figure(figsize=(6,6))
    plt.scatter(y_true, y_pred, alpha=0.4)
    plt.plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 'r--')
    plt.xlabel("실제값")
    plt.ylabel("예측값")
    plt.title(f"실제값 vs 예측값: {model_name}")
    plt.grid()
    plt.show()

    # Residual Plot
    residuals = y_true - y_pred
    plt.figure(figsize=(6,4))
    sns.histplot(residuals, bins=30, kde=True)
    plt.title(f"Residuals Distribution: {model_name}")
    plt.xlabel("Residual")
    plt.ylabel("Count")
    plt.grid()
    plt.show()

def plot_feature_importance(model, feature_names, model_name="Model"):
    try:
        importances = model.feature_importances_
        idx = np.argsort(importances)[::-1]
        plt.figure(figsize=(8,4))
        plt.bar(range(len(importances)), importances[idx])
        plt.xticks(range(len(importances)), np.array(feature_names)[idx], rotation=45)
        plt.title(f'Feature Importance: {model_name}')
        plt.tight_layout()
        plt.show()
    except AttributeError:
        print(f"{model_name}은 feature_importances_ 속성이 없습니다.")

# 6. 모델별 예측 및 평가

# (1) RandomForest
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=120, random_state=42)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)
print_regression_metrics(y_test, rf_pred, "RandomForest")
plot_regression_results(y_test, rf_pred, "RandomForest")
plot_feature_importance(rf, X.columns, "RandomForest")

# (2) XGBoost
from xgboost import XGBRegressor
xgb = XGBRegressor(n_estimators=120, random_state=42)
xgb.fit(X_train, y_train)
xgb_pred = xgb.predict(X_test)
print_regression_metrics(y_test, xgb_pred, "XGBoost")
plot_regression_results(y_test, xgb_pred, "XGBoost")
plot_feature_importance(xgb, X.columns, "XGBoost")

# (3) LightGBM
from lightgbm import LGBMRegressor
lgbm = LGBMRegressor(n_estimators=120, random_state=42)
lgbm.fit(X_train, y_train)
lgbm_pred = lgbm.predict(X_test)
print_regression_metrics(y_test, lgbm_pred, "LightGBM")
plot_regression_results(y_test, lgbm_pred, "LightGBM")
plot_feature_importance(lgbm, X.columns, "LightGBM")

# (4) Stacking 앙상블
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import Ridge

stack = StackingRegressor(
    estimators=[
        ('rf', rf),
        ('xgb', xgb),
        ('lgbm', lgbm)
    ],
    final_estimator=Ridge()
)
stack.fit(X_train, y_train)
stack_pred = stack.predict(X_test)
print_regression_metrics(y_test, stack_pred, "Stacking 앙상블")
plot_regression_results(y_test, stack_pred, "Stacking 앙상블")

# (5) 딥러닝(MLP)
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='linear')
])
model.compile(optimizer='adam', loss='mae', metrics=['mae'])
model.fit(X_train, y_train, validation_split=0.1, epochs=40, batch_size=32, verbose=1)
mlp_pred = model.predict(X_test).flatten()
print_regression_metrics(y_test, mlp_pred, "MLP 딥러닝")
plot_regression_results(y_test, mlp_pred, "MLP 딥러닝")

# (6) 성능 비교표
results = pd.DataFrame({
    'Model': ['RandomForest', 'XGBoost', 'LightGBM', 'Stacking', 'MLP'],
    'MAE': [
        mean_absolute_error(y_test, rf_pred),
        mean_absolute_error(y_test, xgb_pred),
        mean_absolute_error(y_test, lgbm_pred),
        mean_absolute_error(y_test, stack_pred),
        mean_absolute_error(y_test, mlp_pred)
    ],
    'RMSE': [
        np.sqrt(mean_squared_error(y_test, rf_pred)),
        np.sqrt(mean_squared_error(y_test, xgb_pred)),
        np.sqrt(mean_squared_error(y_test, lgbm_pred)),
        np.sqrt(mean_squared_error(y_test, stack_pred)),
        np.sqrt(mean_squared_error(y_test, mlp_pred))
    ],
    'R2': [
        r2_score(y_test, rf_pred),
        r2_score(y_test, xgb_pred),
        r2_score(y_test, lgbm_pred),
        r2_score(y_test, stack_pred),
        r2_score(y_test, mlp_pred)
    ]
})
print('\n\n[모델별 회귀 성능 비교]')
display(results)
